In [1]:
%pip install sentence-transformers

  Using cached pyyaml-6.0.3-cp313-cp313-win_amd64.whl.metadata (2.4 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.0 MB 6.1 MB/s eta 0:00:02
   ------- -------------------------------- 2.4/12.0 MB 6.1 MB/s eta 0:00:02
   ------------ --------------------------- 3.7/12.0 MB 6.2 MB/s eta 0:00:02
   ---------------- ----------------------- 5.0/12.0 MB 6.3 MB/s eta 0:00:02
   -------------------- ------------------- 6.3/12.0 MB 6.3 MB/s eta 0:00:01
   ------------------------- -------------- 7.6/12.0 MB 6.3 MB/s eta 0:00:01
   ----------------------------- ---------- 8.9/12.0 MB 6.4 MB/s eta 0:00:01
   ---------------------------------- ----- 10.2/12.0 MB 6.4 MB/s eta 0:00:01
   -------------------------------------- - 11.5/12.0 MB 6.4 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 6.3 MB/s  0:00:01
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   --------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

sentences = [
    "EUTRAVEL Optimodal European Travel Ecosystem EuTravel aims to:\n1. Support the EU agenda towards an open and single market for mobility services by enabling travellers to organise a multimodal trip in accordance with their own criteria including environmental performance, providing multimodal travel service providers an effective way to deliver customised services addressing any type of specialised travel needs and facilitating fact-based EU policy making.\n2. Promote the creation of content, open and linked data for travellers enriching the travelling experience.\n3. Support travel industry players join forces towards realising an EU shared seamless mobility strategy and architecture.\nEuTravel will research and demonstrate Inter-modal travel optimised with respect to synchronisation between modes, passenger experience and rights and environmental performance (Optimodal Travel). \n\nThe project objectives will be realised by:\n1. Developing an open and readily usable Optimodality Framework aimed at integrating processes, data, and systems in a manner that eliminates interoperability barriers to the marketplace emergence of truly Optimodal travel services: from planning through booking and the full range of related travel support solutions.\n2. Delivering Optimodality Ecosystem Enablers, offering an open infrastructure that allows organisations to set up cost-effective integration of existing systems and to create value added multimodal travel services.\n3. Organising and developing a Living Lab to experiment and evaluate new concepts and prototype solutions in real life multimodal travel scenarios and obtain data to quantify impact.\n4. Taking actions towards sustainable development including a Stakeholder Engagement Strategy and wide dissemination.\nEuTravel, unlike other projects/initiatives, will deliver an Ecosystem promoting and supporting Optimodal travel that will have higher chances of success as it will be populated with tools that tap into existing mainstream IT travel reservation systems and sources of data.",
    "INFORMATION TECHNOLOGIES FOR SHIFT TO RAIL: The IT²RAIL -“Information Technologies for Shift to rail” proposal, first step towards the long term IP4 -“IT for an Attractive Railway” SHIFT²RAIL Innovation Programme, aims at providing a new seamless travel experience, giving access to a complete multimodal travel offer which connects the first and last mile to long distance journeys. This is achieved through the introduction of a ground breaking Technical Enabler based on two concepts: • the traveler is placed at the heart of innovative solutions, accessing all multimodal travel services (shopping, ticketing, and tracking) through its travel-companion. • An open published framework is providing full interoperability whilst limiting impacts on existing systems, without prerequisites for centralized standardization. This Technical Enabler will be completely settled in the context of the SHIFT²RAIL IP4, and IT²RAIL is proposing a reduced approach to the scale of a specified use case without weakening any of the key concepts of IP4, such as the usage of Semantic Web technologies, meta planning on distributed data, travel companion with a protected and secured personal wallet stored in the cloud and including the rights to travel. The use case will be defined as a specific instantiation of our open concepts, and will benefit from a completely scalable architecture fully instantiated in IP4. This approach is addressing all the key challenges of the work program, supporting a complete door-to- door intermodal travel offer and proposing a seamless integration of the very diverse existing and future services for planning, one-stop-shop ticketing, and real-time re-accommodation. Moreover, thanks to an Interoperability framework which insulates travel applications from the standards fragmentation in multimodal transport, IT²RAIL liberates business-model innovations in the market-place, guaranteeing the economic self-sustainability of these e-services in the long-term.",  
    "Hamburg Airport Feasibility Pilot for Seamless Bagshuttle Service: ""Bagshuttle (a brand of Shippies GmbH) plans to deliver baggage from airports, trade fairs or railway stations to hotels or any local destination (and vice versa). Our app-based service wants to give travellers baggage freedom so the customer has the freedom to choose public low-emission mobility (e.g. busses, train) or bicycle, e.g. bike-sharing StadtRAD in Hamburg, instead of cars and taxis to reach their destination. Next to that, business travelers can travel to meetings or fares bag free, tourists can enjoy a sight-seeing tour with their luggage being delivered directly to their hotel. Our vision is to establish an international ""Bagshuttle Alliance"" partnering with airlines, airports, railway corps, trade fairs and hotels to provide bag-free travelling. Airlines for example are expecting additional ancillary services to be chosen by more than half of their passengers in the near future. Bagshuttle addresses every traveller: We won´t be another VIP service for a few people and this is why we think we will have a high impact on how we all travel in the future. The service is highly capable of being integrated in existing sales platforms. The multi-billion travel industry is in the process of digital transformation and our app-based service wants to be part of it. During the program we would start a 4-month pilot at Hamburg Airport for a market survey to get feedback from customers and to find out the logistical challenges. At the end of our feasibility pilot at Hamburg Airport we´ll have a detailed business plan with information regarding market volume, pricing, product characteristics, choice of partners, integrations, market approach and financial planning. Our founder team will make sure that the technical, logistical and financial challenges of this service will be clarified.",     
    "EUTRAVEL Optimodal European Travel Ecosystem EuTravel aims to:\n1. Support the EU agenda towards an open and single market for mobility services by enabling travellers to organise a multimodal trip in accordance with their own criteria including environmental performance, providing multimodal travel service providers an effective way to deliver customised services addressing any type of specialised travel needs and facilitating fact-based EU policy making.\n2. Promote the creation of content, open and linked data for travellers enriching the travelling experience.\n3. Support travel industry players join forces towards realising an EU shared seamless mobility strategy and architecture.\nEuTravel will research and demonstrate Inter-modal travel optimised with respect to synchronisation between modes, passenger experience and rights and environmental performance (Optimodal Travel). \n\nThe project objectives will be realised by:\n1. Developing an open and readily usable Optimodality Framework aimed at integrating processes, data, and systems in a manner that eliminates interoperability barriers to the marketplace emergence of truly Optimodal travel services: from planning through booking and the full range of related travel support solutions.\n2. Delivering Optimodality Ecosystem Enablers, offering an open infrastructure that allows organisations to set up cost-effective integration of existing systems and to create value added multimodal travel services.\n3. Organising and developing a Living Lab to experiment and evaluate new concepts and prototype solutions in real life multimodal travel scenarios and obtain data to quantify impact.\n4. Taking actions towards sustainable development including a Stakeholder Engagement Strategy and wide dissemination.\nEuTravel, unlike other projects/initiatives, will deliver an Ecosystem promoting and supporting Optimodal travel that will have higher chances of success as it will be populated with tools that tap into existing mainstream IT travel reservation systems and sources of data."
]

embeddings = model.encode(sentences)

print(f"Shape των embeddings: {embeddings.shape}") 

score_1_2 = util.cos_sim(embeddings[0], embeddings[1])
score_1_3 = util.cos_sim(embeddings[0], embeddings[2])
score_1_4 = util.cos_sim(embeddings[0], embeddings[3])

print(len(embeddings[0]))

print(f"{score_1_2.item():.4f}")
print(f"{score_1_3.item():.4f}")
print(f"{score_1_4.item():.4f}")


Shape των embeddings: (4, 384)
384
0.6276
0.3828
1.0000
